In [8]:
import ee
ee.Authenticate()
ee.Initialize()


In [9]:
srtm = ee.Image("USGS/SRTMGL1_003")
hydroSHEDS = ee.FeatureCollection("WWF/HydroSHEDS/v1/FreeFlowingRivers")


In [10]:
road_path = ee.Geometry.LineString(
    [[56.0, 31.5], [56.5, 32.0]]  # Replace with actual coordinates
)
buffer_distance = 500  # Distance in meters
study_area = road_path.buffer(buffer_distance)


In [11]:
def fill_sinks(elevation):
    # Simple sink filling algorithm
    filled = elevation.focal_min(3).focal_max(3)
    return filled

# Fill sinks in the elevation data
filled_srtm = fill_sinks(srtm)

# Calculate the flow direction using the D8 algorithm
flow_direction = ee.Algorithms.Hydrology.D8(filled_srtm)

# Calculate the flow accumulation
flow_accumulation = ee.Algorithms.Hydrology.accumulation(flow_direction)

# Threshold to identify streams
stream_threshold = 100
streams = flow_accumulation.gt(stream_threshold)



AttributeError: 

In [ ]:
road_intersections = streams.reduceToVectors(
    geometryType='line',
    reducer=ee.Reducer.countEvery(),
    geometry=study_area,
    scale=30
)

road_crossings = road_intersections.filterBounds(road_path)


NameError: name 'streams' is not defined

In [ ]:
task = ee.batch.Export.table.toDrive(**{
    'collection': road_crossings,
    'description': 'RoadCrossings',
    'folder': 'EarthEngineExports',
    'fileNamePrefix': 'RoadCrossings',
    'fileFormat': 'CSV'
})
task.start()
print("Export task started. Check Google Drive for the file once the task is completed.")


In [12]:
import ee
ee.Initialize()

# Load datasets
srtm = ee.Image("USGS/SRTMGL1_003")

# Define study area and road path
road_path = ee.Geometry.LineString([[56.0, 31.5], [56.5, 32.0]])
buffer_distance = 500  # Distance in meters
study_area = road_path.buffer(buffer_distance)

# Fill sinks in the elevation data
def fill_sinks(elevation):
    filled = elevation.focal_min(3).focal_max(3)
    return filled

filled_srtm = fill_sinks(srtm)

# Compute slope and flow direction
slope = ee.Terrain.slope(filled_srtm)
flow_direction = ee.Terrain.aspect(filled_srtm)

# Calculate flow accumulation
flow_accumulation = filled_srtm.expression(
    '1 + b().multiply(1e6)',
    {'b': filled_srtm}).reduceNeighborhood(
        reducer=ee.Reducer.sum(),
        kernel=ee.Kernel.fixed(3, 3, [[1, 1, 1], [1, 0, 1], [1, 1, 1]])
    )

# Threshold to identify streams
stream_threshold = 100
streams = flow_accumulation.gt(stream_threshold)

# Find road intersections with flow paths
stream_vectors = streams.reduceToVectors(
    geometryType='line',
    reducer=ee.Reducer.countEvery(),
    geometry=study_area,
    scale=30
)

road_crossings = stream_vectors.filterBounds(road_path)

# Export data to Google Drive
task = ee.batch.Export.table.toDrive(**{
    'collection': road_crossings,
    'description': 'RoadCrossings',
    'folder': 'EarthEngineExports',
    'fileNamePrefix': 'RoadCrossings',
    'fileFormat': 'CSV'
})
task.start()

print("Export task started. Check Google Drive for the file once the task is completed.")


Export task started. Check Google Drive for the file once the task is completed.
